In [1]:
# !pip install txtai
# !pip uninstall numpy -y
# !pip install numpy==1.26.4
# !pip install pybind11==2.12.0

In [2]:
import re
from txtai import Embeddings, LLM
from IPython.display import display, HTML
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

In [3]:
# Function to highlight and display code in color
def display_code_color(code):
    formatter = HtmlFormatter(style='colorful')
    highlighted_code = highlight(code, PythonLexer(), formatter)
    style = f"<style>{formatter.get_style_defs()}</style>"
    display(HTML(f"{style}{highlighted_code}"))

# SYSTEM PROMPT

In [4]:
"""
Natapong Nitarach - Typhoon Team Modify From: https://gist.github.com/davidmezzetti/1fac6bd406857431f1cdc74545bdfba9
"""

SYSTEM_PROMPT = {
    "non_self_reflection":"""{SYSTEM}
    You are an AI assistant that uses a Chain of Thought (CoT) approach with reflection to answer queries.
    
    Follow these steps:
    1. Think through the problem step by step within the <thinking> tags.
    2. Reflect on your thinking to check for any errors or improvements within the <reflection> tags.
    3. Make any necessary adjustments based on your reflection.
    4. Provide your final, concise answer within the <output> tags.
    
    Important: The <thinking> sections are for your internal reasoning process only. 
    Do not include any part of the final answer in these sections. 
    The actual response to the query must be entirely contained within the <output> tags.
    
    ### Response Format:
    <thinking>
    [Your step-by-step reasoning goes here. This is your internal thought process, not the final answer.]
    
    <code>
    ```python
    (Optional: functions to solve the question.)
    ```
    </code>
    
    </thinking>
    
    <output>
    "answer": [Your final , concise answer (Thai, English) to the query. This is the only part that will be shown to the user.]
    </output>""",


    
    "self_reflection": """{SYSTEM}
    You are an AI assistant that uses a Chain of Thought (CoT) approach with reflection to answer queries. 
    
    Follow these steps:
    1. Think through the problem step by step within the <thinking> tags.
    2. Reflect on your thinking to check for any errors or improvements within the <reflection> tags.
    3. Reflect within <reflection> tags after key steps. Assign a quality score (0.0-1.0) within <reward> tags to guide adjustments:
        - 0.8+: Continue current approach.
        - 0.5-0.7: Make minor adjustments.
        - Below 0.5: Consider backtracking.
    4. Make any necessary adjustments based on your reflection.
    5. Provide your final, concise answer within the <output> tags.
    
    Important: The <thinking> and <reflection> sections are for your internal reasoning process only. 
    Do not include any part of the final answer in these sections. 
    The actual response to the query must be entirely contained within the <output> tags.
    
    ### Response Format:
    <thinking>
    [Your step-by-step reasoning goes here. This is your internal thought process, not the final answer.]
    
    <code>
    ```python
    (Optional: functions to solve the question.)
    ```
    </code>
    
    <reflection>
    [Your reflection on your reasoning, checking for errors or improvements]
    </reflection>
    [Self-reflection on reasoning quality.]
    <reward>
    [Score between 0.0 and 1.0]
    </reward>
    [Any adjustments to your thinking based on your reflection]
    
    </thinking>
    
    <output>
    "reward": [Your final reflection score],
    "answer": [Your final , concise answer (Thai, English) to the query. This is the only part that will be shown to the user.]
    </output>
    """
    
}

# Functions

In [5]:
def cot(system, user, reflect):

    if reflect:
        system = SYSTEM_PROMPT['non_self_reflection'].format(SYSTEM=system)
    else:
        system = SYSTEM_PROMPT['self_reflection'].format(SYSTEM=system)

    # Run LLM inference
    response = llm([
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ],
        maxlength=4096
    )

    display_code_color(f'Logs of Thought:\n{response}')

    # Extract and return output
    match = re.search(r"<output>(.*?)(?:</output>|$)", response, re.DOTALL)
    return match.group(1).strip() if match else response

In [6]:
def rag(question, reflect=False):
    prompt = """
    Answer the following question using only the context below. Only include information
    specifically discussed.
    question: {question}
    context: {context};
    """

    # System prompt
    system = "You are a friendly assistant. You answer questions from users."

    # RAG context
    context = "\n".join([x["text"] for x in embeddings.search(question)])

    # Control Theory  with CoT with Self-Reflection + RAG
    return cot(system, prompt.format(question=question, context=context), reflect)

In [7]:
# Wikipedia Embeddings Index
embeddings = Embeddings()
embeddings.load(provider="huggingface-hub", container="neuml/txtai-wikipedia")

# LLM
llm = LLM("scb10x/llama-3-typhoon-v1.5x-8b-instruct")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Example: 

## Question-1
A client requires £100,000 one year from now. If the stated annual rate is 2.50% compounded weekly, the deposit needed today is closest to:
#### Options:
- A: £97,500
- B: £97,532 (✅)
- C: £97,561

In [8]:
question = "Read the questions and answers carefully, and choose the one you think is appropriate among the three options A, B and C. Q:A client requires £100,000 one year from now. If the stated annual rate is 2.50% compounded weekly, the deposit needed today is closest to:,CHOICES: A: £97,500.,B: £97,532.,C: £97,561. Answer:"

##### No reflection

In [9]:
result = rag(
    question,
    reflect=False
)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [10]:
display_code_color(result)

##### Reflection

In [11]:
result = rag(
    question,
    reflect=True
)

In [12]:
display_code_color(result)

## Question-2 
กำหนดให้ ABC เป็นรูปสามเหลี่ยมที่มีด้าน AB = AC   AD, BE และ CF เป็นเส้นมัธยฐานที่ลากจากจุด A, B และ C ไปยังด้าน BC, AC และ AB ตามลำดับ AD = 8 หน่วย ผลบวกของกำลังสองของความยาวเส้นมัธยฐานทั้งสามเส้นเท่ากับ 258 จงหาความยาวเส้นรอบรูปของสามเหลี่ยม ABC: 
ข้อใดต่อไปนี้ถูกต้อง 
1.     27  หน่วย
2.     31  หน่วย
3.     32  หน่วย (✅)
4.     36  หน่วย



In [13]:
# ref: https://www.webythebrain.com/article/tu-60_math-test
question = """กำหนดให้ ABC เป็นรูปสามเหลี่ยมที่มีด้าน AB = AC   AD, BE และ CF เป็นเส้นมัธยฐานที่ลากจากจุด A, B และ C ไปยังด้าน BC, AC และ AB ตามลำดับ AD = 8 หน่วย ผลบวกของกำลังสองของความยาวเส้นมัธยฐานทั้งสามเส้นเท่ากับ 258 จงหาความยาวเส้นรอบรูปของสามเหลี่ยม ABC 
ข้อใดต่อไปนี้ถูกต้อง
1. 27  หน่วย
2. 31  หน่วย
3. 32  หน่วย
4. 36  หน่วย
"""

##### No reflection

In [14]:
result = rag(
    question,
    reflect=False
)

In [15]:
display_code_color(result)

##### Reflection

In [16]:
result = rag(
    question, 
    reflect=True
)

In [17]:
display_code_color(result)

## Question-3
1234*2039 เป็นเท่าไหร่? (2,516,126 ✅)



In [18]:
question = "1234*2039 เป็นเท่าไหร่?"

##### No reflection

In [19]:
result = rag(
    question,
    reflect=False
    
)

In [20]:
display_code_color(result)

In [21]:
##### Reflection

In [22]:
result = rag(
    question,
    reflect=True
    
)

In [23]:
display_code_color(result)

## Question-4
ขอสูตรไก่ย่างหน่อย (ของแทร่!!! ของแท้)


In [24]:
question = "ขอสูตรไก่ย่างหน่อย (ของแทร่!!! ของแท้)"
result = rag(question, reflect=True)
display_code_color(result)